# 기본방식으로 연결

In [10]:
import psycopg2
import numpy as np

# 연결
#conn = psycopg2.connect(host='localhost', dbname='postgres', user='admin', password='1234', port=5432)
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

In [11]:
# pgvector 확장
query = """
        CREATE EXTENSION IF NOT EXISTS vector;
        """

cursor.execute(query)

In [12]:
# vector 테이블 생성
cursor.execute("CREATE TABLE IF NOT EXISTS vectors (id bigserial PRIMARY KEY, embedding vector(5));")

In [13]:
# 벡터 생성
example_vectors = np.random.randn(10,5)

# 벡터 삽입
for vec in example_vectors:
    # 벡터 데이터를 삽입 (PostgreSQL의 vector 타입은 문자열 형태로 저장)
    cursor.execute(
        "INSERT INTO vectors (embedding) VALUES (%s)", 
        (vec.tolist(),)  # numpy 배열을 Python 리스트로 변환
    )

# 대량 삽입
vector_data = [(str(vec.tolist()),) for vec in example_vectors] # 벡터 데이터를 문자열로 변환

cursor.executemany(
    "INSERT INTO vectors (embedding) VALUES (%s)",
    vector_data
)

# 커밋
conn.commit()

In [14]:
# 삽입 확인
cursor.execute("SELECT * FROM vectors;")
results = cursor.fetchall()

def show(rows):
    for row in rows:
        print(f"ID: {row[0]}, Vector: {row[1]}")

print('\nAll Rows:')
show(results)


All Rows:
ID: 1, Vector: [0.12701888,0.2993663,-1.2328624,0.6870565,-1.5629419]
ID: 2, Vector: [-0.30376497,-0.0043210364,-0.69145894,0.46969783,0.07878624]
ID: 3, Vector: [0.54155207,-1.3585801,-1.0702776,1.1729195,-0.5504725]
ID: 4, Vector: [-0.23259634,0.77350193,-0.518388,-1.2523943,0.53822637]
ID: 5, Vector: [-0.9098616,-1.2025083,0.47929847,1.6063673,0.610721]
ID: 6, Vector: [-2.4039948,0.14400895,-0.67204183,-1.0767089,0.8748784]
ID: 7, Vector: [-1.2773689,-0.055589672,0.6602447,-1.2963765,0.7178793]
ID: 8, Vector: [-0.49645317,-0.059072237,0.55204207,0.49120075,-0.71522546]
ID: 9, Vector: [-0.18570223,0.8807963,-1.9256159,-0.6219642,-0.20944256]
ID: 10, Vector: [0.45958757,0.7651895,-1.8468606,-0.57813835,0.557521]
ID: 11, Vector: [0.12701888,0.2993663,-1.2328624,0.6870565,-1.5629419]
ID: 12, Vector: [-0.30376497,-0.0043210364,-0.69145894,0.46969783,0.07878624]
ID: 13, Vector: [0.54155207,-1.3585801,-1.0702776,1.1729195,-0.5504725]
ID: 14, Vector: [-0.23259634,0.77350193,-0.51

In [15]:
np.random.randn(1, 5).tolist()

[[0.8682892366420777,
  2.4796360452916373,
  0.7687134195230758,
  -0.4908894881961153,
  -1.4130724736305247]]

In [16]:
%timeit
test_vec = np.random.randn(1,5)
test_vec_list = test_vec.tolist()[0]
cursor.execute("SELECT * FROM vectors ORDER BY embedding <-> '{}' LIMIT 5".format(test_vec_list))
results = cursor.fetchall()
show(results)

ID: 13, Vector: [0.54155207,-1.3585801,-1.0702776,1.1729195,-0.5504725]
ID: 3, Vector: [0.54155207,-1.3585801,-1.0702776,1.1729195,-0.5504725]
ID: 1, Vector: [0.12701888,0.2993663,-1.2328624,0.6870565,-1.5629419]
ID: 11, Vector: [0.12701888,0.2993663,-1.2328624,0.6870565,-1.5629419]
ID: 2, Vector: [-0.30376497,-0.0043210364,-0.69145894,0.46969783,0.07878624]


In [17]:
# 연결 종료
cursor.close()
conn.close()